In [1]:
import pandas as pd
import requests
import re
from datetime import datetime as dt
import itertools
import time
import logging
logging.basicConfig(filename='logs.log', 
                    filemode='w',
                    level=logging.INFO, 
                    format='%(levelname)s: (%(asctime)s) %(message)s', 
                    datefmt='%m/%d/%Y %H:%M:%S')

In [15]:
year_list = range(last_year, first_year-1, -1)
list(year_list)


[2017,
 2016,
 2015,
 2014,
 2013,
 2012,
 2011,
 2010,
 2009,
 2008,
 2007,
 2006,
 2005,
 2004,
 2003,
 2002]

In [2]:
%%capture
from tqdm.notebook import tqdm
tqdm().pandas()

In [4]:
# There are text files containing volume counts available from the MnDOT website. They are on the main site, but the index
# page is no longer accessible. An archived index page is available on the WaybackMachine, located at
# https://web.archive.org/web/20181117175447/http://www.dot.state.mn.us:80/traffic/data/reports-hrvol-atr.html
archived_station_list = [8, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 38, 39, 40, 41, 42, 43, 44, 45, 46, 48, 49, 51, \
                         53, 54, 55, 56, 57, 101, 102, 103, 110, 164, 170, 172, 175, 179, 187, 188, 191, 195, 197, 198, \
                         199, 200, 204, 208, 209, 210, 211, 212, 213, 214, 218, 219, 220, 221, 222, 223, 225, 227, 228, \
                         229, 230, 231, 232, 233, 301, 303, 305, 309, 315, 321, 326, 329, 335, 336, 341, 342, 351, 352, \
                         353, 354, 359, 365, 381, 382, 384, 386, 388, 389, 390, 400, 402, 405, 407, 410, 420, 422, 425, \
                         458, 460, 464]

# The files themselves are named https://www.dot.state.mn.us/traffic/data/reports/atr/Hourly_Volume/YYYY/ATRxxx.txt
# where YYYY is the four-digit year (between 2002 and 2017) and xxx is the station id number (left-padded with 0 if the 
# id has fewer than three digits.) If the year is 2010 or earlier, the file suffix must be capitalized (".TXT" instead of
# ".txt")
first_year = 2002
last_year = 2017
url_stem = 'https://www.dot.state.mn.us/traffic/data/reports/atr/Hourly_Volume/'
file_suffixes_pre = { i: ".TXT" for i in range(first_year, 2011) }
file_suffixes_post = { i: ".txt" for i in range(2011, last_year + 1) }
file_suffix = { **file_suffixes_pre, **file_suffixes_post }

directions = { 'North': 1, 'East': 3, 'South': 5, 'West': 7}
# Part of one month for station 231 uses the directions "northeast" and "southwest" rather than "east" and "west".
# Substitute the values for "east" and "west" when this arises.
irregular_directions = { ('Northeast', 231): 3, ('Southwest', 231): 7 }

In [5]:
# Each page of the text reports will contain a header line indicating the station number, the direction, and the month.
# The station number and month are redundant, but the direction is needed.
state_pattern = re.compile(r'Station\s*(\d+)\,\s*Direction\s*(\w+)')

# The data lines will start with a date, although there may be a star before the line. If the star is present, it means
# that some or all of the data in the row is an estimate.
row_pattern = re.compile(r'^(\**)\s*(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)')
data_pattern = re.compile(r'\d+')

# The names and order of columns are meant to match with those in the more recent (2017-present) csv files.
columns = ['station_id', 'dir_of_travel', 'lane_of_travel', 'date'] + list(map(str, range(1, 25)))

In [19]:
def process_response(station_id, response):
    
    logging.info(f'Year:{year}, station_id:{station_id} processing')
    count_data = {}
    
    for line in response.iter_lines():
        # Without decoding the line, reg ex searches will raise an error
        line = line.decode('utf-8')

        # Check for the header line. If we have the correct line, extract the corresponding direction and move to the
        # next line.
        if state_match := state_pattern.search(line):
            # Check that anticipated station id is correct. 
            try:
                assert station_id == int(state_match.group(1))
            except AssertionError:
                logging.warning(f'Year:{year}, station_id:{station_id} corrected to {int(state_match.group(1))}')
                station_id = int(state_match.group(1))
                
            try:
                # Move to direction information.
                dir_state_str = state_match.group(2)
                # Guard against possibility that capitalization of directions changes between files.
                dir_state_val = directions[dir_state_str.title()]
            except Exception:
                logging.warning(f'Year: {year}, station_id: {station_id}, irregular direction {dir_state_str} seen')
                dir_state_val = irregular_directions[(dir_state_str.title(), station_id)]
                
            continue

        # Check for data line. For matches, extract the date and hourly counts
        if row_match := row_pattern.search(line):
            # row_match will match any * at the beginning as well as the month abbreviation. data_match will match any
            # numbers. The first two numbers will be the day and year, so these are combined with the second row match 
            # group to get the date.
            data_match = data_pattern.findall(line)
            date_list = [row_match.group(2)] + data_match[:2]
            date_str = ' '.join(date_list)
            date = dt.strptime(date_str, '%b %d %Y')

            # The remaining numeric data are the hourly counts and the daily count. Only the hourly counts are needed, but
            # the daily count is used as a check.
            hourly_list = data_match[2:-1]
            
            try:
                assert len(hourly_list) == 24
            except AssertionError:
                logging.warning(f'Date: {date}, station_id: {station_id}, direction: {dir_state_val} hour count mismatch')
                continue
                
            hourly_counts = list(map(int, hourly_list))
            daily_total = int(data_match[-1])
            
            try:
                assert sum(hourly_counts) == daily_total
            except AssertionError:
#                 logging.warning(f'Date: {date}, station_id: {station_id}, direction: {dir_state_val} daily total mismatch')
                pass

            # Combine the station id and direction value with the counts in a single list to add into a dictionary keyed 
            # by the date.
            # The idea is that this will be a single row of a data frame.
            # The 0 is for lane of travel, which is present in newer csv files, but not in the older text outputs.
            # The coding as 0 matches the csv fles without lane info.
            data_list = [station_id, dir_state_val, 0, date] + hourly_counts
            count_data[len(count_data)] = dict(zip(columns, data_list))
            
    # Convert count columns to integer type. station_id, dir_of_travel, and lane_of_travel are all provided numerically
    # but are functionally categorical.
    df = pd.DataFrame(count_data).T
    
    return df

In [28]:
text_file_name = 'Data/text_data.pkl'
try:
    text_data = pd.read_pickle(text_file_name)
except IOError:
    logging.warning(f'Error reading {text_file_name}')
        
    df_list = []
    year_list = range(last_year, first_year-1, -1)
    it_list = list(itertools.product(*[archived_station_list, year_list]))

    for station_no, year in tqdm(it_list):
        url = url_stem + str(year) + '/' + 'ATR' + str(station_no).rjust(3, "0") + file_suffix[year]
        redo = True

        while redo:
            try:
                response = requests.get(url)
                if response.status_code == 200:
                    df_list.append(process_response(station_no, response))
                else:
#                     logging.warning(f'Year: {year}, station_id: {station_no} gives status code {response.status_code}')
                    pass
                redo = False
            except requests.exceptions.RequestException as e:
                print(f'Error {e} reading page {url}')
            except Exception:
                print(f'Unexpected Error')
                raise

    text_data = pd.concat(df_list)
    
    # Station 53 was duplicated in 2007 (Station 51 information is erroneous).
    text_data_dups = text_data.duplicated(keep='first')
    text_data[text_data_dups]
    text_data = text_data[~text_data_dups]
    
    text_data.to_pickle(text_file_name)
    


In [29]:
# The 2017 data comes in two formats - the text files processed above and the csv files. Since it is already included,
# ignore the 2017 csv file.
csv_files = ['2018_wim_atr_volume-10913555-v1.csv', '2019_wim_atr_volume-10913553-v1.csv', \
             '2020_wim_atr_volume-10913551-v7.csv']
xlsx_files = ['2021_wim_atr_volume.xlsx']

In [30]:
# 2021 is stored in an Excel workbook rather than a csv file and also uses integers rather than strings for column names. 
dfs = [text_data] + [pd.read_csv('Data/' + file_name, parse_dates=['date']) for file_name in csv_files] \
+ [pd.read_excel('Data/' + file_name, parse_dates=['date'], header=0, names=columns) for file_name in xlsx_files]
traffic_volume_df = pd.concat(dfs)

In [32]:
pd.set_option('display.max_columns', None)
traffic_volume_df

,station_id,dir_of_travel,lane_of_travel,date,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24
0,8,3,0,2017-01-01,0,3,1,1,0,0,2,2,2,1,0,12,9,8,4,8,5,4,6,6,3,2,1,0
1,8,3,0,2017-01-02,0,0,1,0,0,0,2,7,8,5,7,11,8,6,15,7,8,6,10,4,4,5,1,0
2,8,3,0,2017-01-03,0,0,1,0,0,0,5,8,4,9,18,19,16,16,11,17,12,9,5,10,4,1,0,0
3,8,3,0,2017-01-04,1,0,1,0,0,0,5,4,8,27,15,10,15,9,10,24,14,20,18,7,1,2,2,0
4,8,3,0,2017-01-05,0,0,1,0,1,1,5,20,11,10,8,13,10,11,9,14,16,34,27,10,2,7,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11025,6224,7,2,2021-02-26,6,2,3,0,2,4,10,5,14,16,9,10,16,17,13,22,17,9,11,18,16,7,7,2
11026,6224,3,1,2021-02-27,52,57,39,41,38,42,54,109,179,286,325,319,272,311,329,293,323,275,280,207,203,154,84,64
11027,6224,3,2,2021-02-27,9,5,10,3,4,6,12,21,39,92,109,101,106,86,102,88,96,86,73,56,47,36,16,11
11028,6224,7,1,2021-02-27,86,64,42,37,53,48,76,108,163,257,269,359,364,317,310,290,249,267,213,235,224,165,130,94


In [33]:
traffic_volume_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1115516 entries, 0 to 11029
Data columns (total 28 columns):
 #   Column          Non-Null Count    Dtype         
---  ------          --------------    -----         
 0   station_id      1115516 non-null  object        
 1   dir_of_travel   1115516 non-null  object        
 2   lane_of_travel  1115516 non-null  object        
 3   date            1115516 non-null  datetime64[ns]
 4   1               1115516 non-null  object        
 5   2               1115516 non-null  object        
 6   3               1115516 non-null  object        
 7   4               1115516 non-null  object        
 8   5               1115516 non-null  object        
 9   6               1115516 non-null  object        
 10  7               1115516 non-null  object        
 11  8               1115516 non-null  object        
 12  9               1115516 non-null  object        
 13  10              1115516 non-null  object        
 14  11              1115

In [34]:
non_numeric_columns = ['station_id', 'dir_of_travel', 'lane_of_travel', 'date']
traffic_volume_df.loc[:, ~traffic_volume_df.columns.isin(non_numeric_columns)] \
        = traffic_volume_df.loc[:, ~traffic_volume_df.columns.isin(non_numeric_columns)].apply(pd.to_numeric)

In [35]:
traffic_volume_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1115516 entries, 0 to 11029
Data columns (total 28 columns):
 #   Column          Non-Null Count    Dtype         
---  ------          --------------    -----         
 0   station_id      1115516 non-null  object        
 1   dir_of_travel   1115516 non-null  object        
 2   lane_of_travel  1115516 non-null  object        
 3   date            1115516 non-null  datetime64[ns]
 4   1               1115516 non-null  int64         
 5   2               1115516 non-null  int64         
 6   3               1115516 non-null  int64         
 7   4               1115516 non-null  int64         
 8   5               1115516 non-null  int64         
 9   6               1115516 non-null  int64         
 10  7               1115516 non-null  int64         
 11  8               1115516 non-null  int64         
 12  9               1115516 non-null  int64         
 13  10              1115516 non-null  int64         
 14  11              1115

In [36]:
station_list = traffic_volume_df.station_id.unique()
station_list.sort()
station_category = pd.CategoricalDtype(categories=station_list, ordered=True)
traffic_volume_df.loc[:, 'station_id'] = traffic_volume_df.loc[:, 'station_id'].astype(station_category)

direction_list = traffic_volume_df.dir_of_travel.unique()
direction_list.sort()
direction_category = pd.CategoricalDtype(categories=direction_list, ordered=True)
traffic_volume_df.loc[:, 'dir_of_travel'] = traffic_volume_df.loc[:, 'dir_of_travel'].astype(direction_category)

In [17]:
traffic_volume_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 877872 entries, 0 to 11029
Data columns (total 28 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   station_id      877872 non-null  category      
 1   dir_of_travel   877872 non-null  category      
 2   lane_of_travel  877872 non-null  object        
 3   date            877872 non-null  datetime64[ns]
 4   1               877872 non-null  int64         
 5   2               877872 non-null  int64         
 6   3               877872 non-null  int64         
 7   4               877872 non-null  int64         
 8   5               877872 non-null  int64         
 9   6               877872 non-null  int64         
 10  7               877872 non-null  int64         
 11  8               877872 non-null  int64         
 12  9               877872 non-null  int64         
 13  10              877872 non-null  int64         
 14  11              877872 non-null  int6

In [37]:
traffic_volume_df

,station_id,dir_of_travel,lane_of_travel,date,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24
0,8,3,0,2017-01-01,0,3,1,1,0,0,2,2,2,1,0,12,9,8,4,8,5,4,6,6,3,2,1,0
1,8,3,0,2017-01-02,0,0,1,0,0,0,2,7,8,5,7,11,8,6,15,7,8,6,10,4,4,5,1,0
2,8,3,0,2017-01-03,0,0,1,0,0,0,5,8,4,9,18,19,16,16,11,17,12,9,5,10,4,1,0,0
3,8,3,0,2017-01-04,1,0,1,0,0,0,5,4,8,27,15,10,15,9,10,24,14,20,18,7,1,2,2,0
4,8,3,0,2017-01-05,0,0,1,0,1,1,5,20,11,10,8,13,10,11,9,14,16,34,27,10,2,7,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11025,6224,7,2,2021-02-26,6,2,3,0,2,4,10,5,14,16,9,10,16,17,13,22,17,9,11,18,16,7,7,2
11026,6224,3,1,2021-02-27,52,57,39,41,38,42,54,109,179,286,325,319,272,311,329,293,323,275,280,207,203,154,84,64
11027,6224,3,2,2021-02-27,9,5,10,3,4,6,12,21,39,92,109,101,106,86,102,88,96,86,73,56,47,36,16,11
11028,6224,7,1,2021-02-27,86,64,42,37,53,48,76,108,163,257,269,359,364,317,310,290,249,267,213,235,224,165,130,94


In [38]:
traffic_volume_df = pd.melt(traffic_volume_df, id_vars=['station_id', 'dir_of_travel', 'lane_of_travel', 'date'], 
        var_name='hour', value_name='count')

In [39]:
traffic_volume_df['datetime'] = traffic_volume_df['date'] \
    + pd.to_timedelta(traffic_volume_df['hour'].astype(int) - 1, unit='h')

In [40]:
traffic_volume_df.drop(['date', 'hour'], axis=1, inplace=True)

In [41]:
# Fold the lane of travel into the columns.
traffic_volume_df = pd.concat([traffic_volume_df[['station_id', 'dir_of_travel', 'datetime']], 
                               traffic_volume_df.pivot(columns='lane_of_travel', values='count')], axis=1)

In [42]:
traffic_volume_df.columns = ['station_id', 'dir_of_travel', 'datetime', 'lane_x', 'lane_1', 'lane_2', 'lane_3', 'lane_4']

In [43]:
traffic_volume_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26772384 entries, 0 to 26772383
Data columns (total 8 columns):
 #   Column         Dtype         
---  ------         -----         
 0   station_id     category      
 1   dir_of_travel  category      
 2   datetime       datetime64[ns]
 3   lane_x         float64       
 4   lane_1         float64       
 5   lane_2         float64       
 6   lane_3         float64       
 7   lane_4         float64       
dtypes: category(2), datetime64[ns](1), float64(5)
memory usage: 1.2 GB


In [44]:
traffic_volume_df

,station_id,dir_of_travel,datetime,lane_x,lane_1,lane_2,lane_3,lane_4
0,8,3,2017-01-01 00:00:00,0.0,NaN,NaN,NaN,NaN
1,8,3,2017-01-02 00:00:00,0.0,NaN,NaN,NaN,NaN
2,8,3,2017-01-03 00:00:00,0.0,NaN,NaN,NaN,NaN
3,8,3,2017-01-04 00:00:00,1.0,NaN,NaN,NaN,NaN
4,8,3,2017-01-05 00:00:00,0.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
26772379,6224,7,2021-02-26 23:00:00,NaN,NaN,2.0,NaN,NaN
26772380,6224,3,2021-02-27 23:00:00,NaN,64.0,NaN,NaN,NaN
26772381,6224,3,2021-02-27 23:00:00,NaN,NaN,11.0,NaN,NaN
26772382,6224,7,2021-02-27 23:00:00,NaN,94.0,NaN,NaN,NaN


In [46]:
traffic_volume_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26772384 entries, 0 to 26772383
Data columns (total 8 columns):
 #   Column         Dtype         
---  ------         -----         
 0   station_id     category      
 1   dir_of_travel  category      
 2   datetime       datetime64[ns]
 3   lane_x         float64       
 4   lane_1         float64       
 5   lane_2         float64       
 6   lane_3         float64       
 7   lane_4         float64       
dtypes: category(2), datetime64[ns](1), float64(5)
memory usage: 1.2 GB


In [47]:
traffic_volume_df.describe()

,lane_x,lane_1,lane_2,lane_3,lane_4
count,2.343518e+07,2.141664e+06,1.069440e+06,96744.000000,29352.000000
mean,6.149396e+02,1.633071e+02,1.569376e+02,155.771211,203.532093
std,1.045198e+03,2.106097e+02,2.234375e+02,197.594621,223.703095
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000
25%,3.100000e+01,1.800000e+01,1.200000e+01,11.000000,8.000000
50%,1.640000e+02,7.400000e+01,5.200000e+01,60.000000,117.000000
75%,7.040000e+02,2.350000e+02,2.280000e+02,266.000000,362.000000
max,1.029000e+04,5.196000e+03,3.841000e+03,1841.000000,1636.000000


In [48]:
traffic_volume_df = traffic_volume_df.groupby(['station_id', 'dir_of_travel', 'datetime'], 
                                              as_index=False, sort=False, observed=True).sum()

In [49]:
traffic_volume_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25576848 entries, 0 to 25576847
Data columns (total 8 columns):
 #   Column         Dtype         
---  ------         -----         
 0   station_id     category      
 1   dir_of_travel  category      
 2   datetime       datetime64[ns]
 3   lane_x         float64       
 4   lane_1         float64       
 5   lane_2         float64       
 6   lane_3         float64       
 7   lane_4         float64       
dtypes: category(2), datetime64[ns](1), float64(5)
memory usage: 1.4 GB


In [50]:
traffic_volume_df

,station_id,dir_of_travel,datetime,lane_x,lane_1,lane_2,lane_3,lane_4
0,8,3,2017-01-01 00:00:00,0.0,0.0,0.0,0.0,0.0
1,8,3,2017-01-02 00:00:00,0.0,0.0,0.0,0.0,0.0
2,8,3,2017-01-03 00:00:00,0.0,0.0,0.0,0.0,0.0
3,8,3,2017-01-04 00:00:00,1.0,0.0,0.0,0.0,0.0
4,8,3,2017-01-05 00:00:00,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
25576843,6224,7,2021-02-25 23:00:00,0.0,103.0,4.0,0.0,0.0
25576844,6224,3,2021-02-26 23:00:00,0.0,81.0,26.0,0.0,0.0
25576845,6224,7,2021-02-26 23:00:00,0.0,100.0,2.0,0.0,0.0
25576846,6224,3,2021-02-27 23:00:00,0.0,64.0,11.0,0.0,0.0


In [51]:
traffic_volume_df['total'] = traffic_volume_df.iloc[:, -5:-1].sum(axis=1)

In [52]:
traffic_volume_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25576848 entries, 0 to 25576847
Data columns (total 9 columns):
 #   Column         Dtype         
---  ------         -----         
 0   station_id     category      
 1   dir_of_travel  category      
 2   datetime       datetime64[ns]
 3   lane_x         float64       
 4   lane_1         float64       
 5   lane_2         float64       
 6   lane_3         float64       
 7   lane_4         float64       
 8   total          float64       
dtypes: category(2), datetime64[ns](1), float64(6)
memory usage: 1.6 GB


In [53]:
traffic_volume_df.describe()

,lane_x,lane_1,lane_2,lane_3,lane_4,total
count,2.557685e+07,2.557685e+07,2.557685e+07,2.557685e+07,2.557685e+07,2.557685e+07
mean,5.634480e+02,1.367444e+01,6.562003e+00,5.892020e-01,2.335735e-01,5.842736e+02
std,1.014878e+03,7.589667e+01,5.544583e+01,1.546331e+01,1.024269e+01,1.011423e+03
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.700000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.000000e+01
50%,1.240000e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.550000e+02
75%,6.190000e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,6.600000e+02
max,1.029000e+04,5.196000e+03,3.841000e+03,1.841000e+03,1.636000e+03,1.029000e+04


In [54]:
non_numeric_columns = ['station_id', 'dir_of_travel', 'datetime']
traffic_volume_df.loc[:, ~traffic_volume_df.columns.isin(non_numeric_columns)] \
        = traffic_volume_df.loc[:, ~traffic_volume_df.columns.isin(non_numeric_columns)].astype('int16')

In [55]:
traffic_volume_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25576848 entries, 0 to 25576847
Data columns (total 9 columns):
 #   Column         Dtype         
---  ------         -----         
 0   station_id     category      
 1   dir_of_travel  category      
 2   datetime       datetime64[ns]
 3   lane_x         int16         
 4   lane_1         int16         
 5   lane_2         int16         
 6   lane_3         int16         
 7   lane_4         int16         
 8   total          int16         
dtypes: category(2), datetime64[ns](1), int16(6)
memory usage: 731.8 MB


In [56]:
traffic_volume_df = traffic_volume_df.sort_values(['station_id', 'dir_of_travel', 'datetime'])
traffic_volume_df.reset_index(drop=True, inplace=True)

In [57]:
traffic_volume_df

,station_id,dir_of_travel,datetime,lane_x,lane_1,lane_2,lane_3,lane_4,total
0,8,1,2012-07-02 00:00:00,0,0,0,0,0,0
1,8,1,2012-07-02 01:00:00,0,0,0,0,0,0
2,8,1,2012-07-02 02:00:00,0,0,0,0,0,0
3,8,1,2012-07-02 03:00:00,2,0,0,0,0,2
4,8,1,2012-07-02 04:00:00,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
25576843,6224,7,2021-02-27 19:00:00,0,235,7,0,0,242
25576844,6224,7,2021-02-27 20:00:00,0,224,6,0,0,230
25576845,6224,7,2021-02-27 21:00:00,0,165,3,0,0,168
25576846,6224,7,2021-02-27 22:00:00,0,130,1,0,0,131


In [58]:
volume_file_name = 'Data/volume_data.pkl'
traffic_volume_df.to_pickle(volume_file_name)